# Combining Different Models for Ensemble Learning

https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch07/ch07.ipynb

In [15]:
# Added version check for recent scikit-learn 0.18 checks
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version
#from scipy.misc import comb
from scipy.special import comb
    
import math
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import sklearn
import utils

np.random.seed(123)

def ensemble_error(n_classifier, error):
    k_start = math.ceil(n_classifier / 2.0)
    probs = [comb(n_classifier, k) * error**k * (1-error)**(n_classifier - k)
             for k in range(k_start, n_classifier + 1)]
    return sum(probs)


# Load normalized data

In [21]:
# train and test datasets
#train x
train_reviews = utils.readFromDisk('train_reviews')
#train y
train_sentiments = utils.readFromDisk('train_sentiments')
#text x
test_reviews = utils.readFromDisk('test_reviews')
#test y
test_sentiments = utils.readFromDisk('test_sentiments')
#tokenized train
tokenized_train = utils.readFromDisk('tokenized_train')
#tokenized test
tokenized_test= utils.readFromDisk('tokenized_test')
#onehotencoded train sentiments, encoded by labelencoder
train_sentiments_encoded = utils.readFromDisk('train_sentiments_encoded')
#onehotencoded test sentiments, encoded by labelencoder
test_sentiments_encoded = utils.readFromDisk('test_sentiments_encoded')

#glove features
# feature engineering with GloVe model
train_glove_features = utils.readFromDisk('train_glove_features')
test_glove_features = utils.readFromDisk('test_glove_features')



#print('train_reviews:',train_reviews)
#print('train_sentiments:',train_sentiments)
#print('test_reviews:',test_reviews)
#print('test_sentiments:',test_sentiments)

#print('tokenized_train:',tokenized_train)
#print('tokenized_test:',tokenized_test)

#print('train_sentiments_encoded:',train_sentiments_encoded)
#print('test_sentiments_encoded:',test_sentiments_encoded)

#print('train_glove_features:',train_glove_features)
#print('test_glove_features:',test_glove_features)

train_glove_features: [[-0.06544407  0.17556535 -0.21013008 ... -0.04701199  0.00197406
   0.16638027]
 [-0.04093199  0.1982432  -0.13336308 ... -0.05149595 -0.04492578
   0.16158599]
 [-0.05953808  0.21305391 -0.19624065 ... -0.06175661  0.00524534
   0.1403496 ]
 ...
 [-0.03195357  0.2017131  -0.17966323 ... -0.03751112 -0.00171579
   0.12789251]
 [-0.00740553  0.2523542  -0.1440551  ... -0.03443174 -0.03967679
   0.14882289]
 [-0.09199697  0.22967067 -0.12079366 ... -0.03300412 -0.0551823
   0.16499974]]


In [23]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()

print('5-fold cross validation:\n')

for clf, label in zip([clf1, clf2, clf3], ['Logistic Regression', 'Random Forest', 'naive Bayes']):

    #print('train_sentiments_encoded:',train_sentiments_encoded)
    #print('train_sentiments:',train_sentiments)
    
    scores = model_selection.cross_val_score(clf, train_glove_features, train_sentiments, cv=5, scoring='accuracy')
    print("Accuracy: ", scores.mean(), scores.std(), label)


5-fold cross validation:



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressio

Accuracy:  0.898362669397458 0.0030768234470832673 Logistic Regression
Accuracy:  0.8586825773144182 0.0036945978633773683 Random Forest
Accuracy:  0.81103904875455 0.0028661328521687205 naive Bayes


# Implementing a simple majority vote classifier

Now, we will implement a simple EnsembleClassifier class that allows us to combine the three different classifiers. We define a predict method that let's us simply take the majority rule of the predictions by the classifiers. E.g., if the prediction for a sample is

- classifier 1 -> class 1
- classifier 2 -> class 1
- classifier 3 -> class 2
we would classify the sample as "class 1."

Furthermore, we add a weights parameter, which let's us assign a specific weight to each classifier. In order to work with the weights, we collect the predicted class probabilities for each classifier, multiply it by the classifier weight, and take the average. Based on these weighted average probabilties, we can then assign the class label.

To illustrate this with a simple example, let's assume we have 3 classifiers and a 3-class classification problems where we assign equal weights to all classifiers (the default): w1=1, w2=1, w3=1.

The weighted average probabilities for a sample would then be calculated as follows:

classifier	| class 1	| class 2	| class 3
------------|------------|------------|---------:
classifier 1	| w1 * 0.2	| w1 * 0.5	| w1 * 0.3
classifier 2	| w2 * 0.6	| w2 * 0.3	| w2 * 0.1
classifier 3	| w3 * 0.3	| w3 * 0.4	| w3 * 0.3
weighted average	| 0.37	| 0.4	| 0.3
We can see in the table above that class 2 has the highest weighted average probability, thus we classify the sample as class 2.

Now, let's put it into code and apply it to our Iris classification.




In [24]:
import numpy as np

np.argmax(np.bincount([0, 0, 1], 
                      weights=[0.2, 0.2, 0.6]))


1

In [25]:
ex = np.array([[0.9, 0.1],
               [0.8, 0.2],
               [0.4, 0.6]])

p = np.average(ex, 
               axis=0, 
               weights=[0.2, 0.2, 0.6])
p


array([0.58, 0.42])

In [26]:
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator


class MajorityVoteClassifier(BaseEstimator, 
                             ClassifierMixin):
    """ A majority vote ensemble classifier

    Parameters
    ----------
    classifiers : array-like, shape = [n_classifiers]
      Different classifiers for the ensemble

    vote : str, {'classlabel', 'probability'} (default='label')
      If 'classlabel' the prediction is based on the argmax of
        class labels. Else if 'probability', the argmax of
        the sum of probabilities is used to predict the class label
        (recommended for calibrated classifiers).

    weights : array-like, shape = [n_classifiers], optional (default=None)
      If a list of `int` or `float` values are provided, the classifiers
      are weighted by importance; Uses uniform weights if `weights=None`.

    """
    def __init__(self, classifiers, vote='classlabel', weights=None):

        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value
                                  in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights

    def fit(self, X, y):
        """ Fit classifiers.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.

        y : array-like, shape = [n_samples]
            Vector of target class labels.

        Returns
        -------
        self : object

        """
        if self.vote not in ('probability', 'classlabel'):
            raise ValueError("vote must be 'probability' or 'classlabel'"
                             "; got (vote=%r)"
                             % self.vote)

        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d classifiers'
                             % (len(self.weights), len(self.classifiers)))

        # Use LabelEncoder to ensure class labels start with 0, which
        # is important for np.argmax call in self.predict
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self

    def predict(self, X):
        """ Predict class labels for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Matrix of training samples.

        Returns
        ----------
        maj_vote : array-like, shape = [n_samples]
            Predicted class labels.
            
        """
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1)
        else:  # 'classlabel' vote

            #  Collect results from clf.predict calls
            predictions = np.asarray([clf.predict(X)
                                      for clf in self.classifiers_]).T

            maj_vote = np.apply_along_axis(
                                      lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions)
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote

    def predict_proba(self, X):
        """ Predict class probabilities for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.

        Returns
        ----------
        avg_proba : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.

        """
        probas = np.asarray([clf.predict_proba(X)
                             for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba

    def get_params(self, deep=True):
        """ Get classifier parameter names for GridSearch"""
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in six.iteritems(self.named_classifiers):
                for key, value in six.iteritems(step.get_params(deep=True)):
                    out['%s__%s' % (name, key)] = value
            return out


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


## Combining different algorithms for classification with majority vote


In [34]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split

#iris = datasets.load_iris()
#X, y = iris.data[50:, [1, 2]], iris.target[50:]

X, y = train_glove_features, train_sentiments
le = LabelEncoder()
y = le.fit_transform(y)


#X_train, X_test, y_train, y_test = test_glove_features, train_glove_features, train_sentiments, test_sentiments

X_train, X_test, y_train, y_test =\
       train_test_split(X, y, 
                        test_size=0.5, 
                        random_state=1)


In [35]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import cross_val_score
else:
    from sklearn.model_selection import cross_val_score

clf1 = LogisticRegression(penalty='l2', 
                          C=0.001,
                          random_state=0)

clf2 = DecisionTreeClassifier(max_depth=1,
                              criterion='entropy',
                              random_state=0)

clf3 = KNeighborsClassifier(n_neighbors=1,
                            p=2,
                            metric='minkowski')

pipe1 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf1]])
pipe3 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf3]])

clf_labels = ['Logistic Regression', 'Decision Tree', 'KNN']

print('10 fold cross validation:\n')

for clf, label in zip([pipe1, clf2, pipe3], clf_labels):
    scores = model_selection.cross_val_score(estimator=clf,
                             X=X_train,
                             y=y_train,
                             cv=10,
                             scoring='roc_auc')
    print("ROC AUC", (scores.mean(), scores.std(), label))


10 fold cross validation:

ROC AUC (0.955541189681858, 0.0021592498247352634, 'Logistic Regression')
ROC AUC (0.6990416527900465, 0.007421903939988239, 'Decision Tree')
ROC AUC (0.764596489050351, 0.0032448613901172943, 'KNN')


In [36]:
# Majority Rule (hard) Voting

mv_clf = MajorityVoteClassifier(classifiers=[pipe1, clf2, pipe3])

clf_labels += ['Majority Voting']
all_clf = [pipe1, clf2, pipe3, mv_clf]

for clf, label in zip(all_clf, clf_labels):
    scores = cross_val_score(estimator=clf,
                             X=X_train,
                             y=y_train,
                             cv=10,
                             scoring='roc_auc')
    print("ROC AUC: %0.2f (+/- %0.2f) [%s]"
          % (scores.mean(), scores.std(), label))


ROC AUC: 0.96 (+/- 0.00) [Logistic Regression]
ROC AUC: 0.70 (+/- 0.01) [Decision Tree]


KeyboardInterrupt: 